In [1]:
!nvidia-smi

Sun Jun  4 21:47:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti    WDDM | 00000000:2B:00.0  On |                  N/A |
|  0%   43C    P8               14W / 290W|   1172MiB /  8192MiB |     14%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install datasets transformers rouge_score


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 126.5/126.5 kB 2.5 MB/s eta 0:00:00
  Running setup.py install for rouge_score: started
  Running setup.py install for rouge_score: finished with status 'done'


  DEPRECATION: rouge_score is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import LEDForConditionalGeneration, LEDTokenizer

c:\Users\win10\Documents\podnotes-qa\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = LEDTokenizer.from_pretrained("pszemraj/led-large-book-summary")
model = LEDForConditionalGeneration.from_pretrained("pszemraj/led-large-book-summary").to("cuda").half()

In [8]:
import torch

def generate_answer(batch):
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=8384, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")

  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, max_length=8112, num_beams=4)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch


In [10]:
import os
print(os.getcwd())


with open('episode.txt', 'r') as file:
        contents = file.read()
        batch = {"article": contents}

        result = generate_answer(batch)
        print(result['predicted_abstract'])


c:\Users\win10\Documents\podnotes-qa\workspace
['On his third appearance on the podcast, Dr. Barlow speaks with Max Tegmark about artificial intelligence and why it\'s important to have the right kind of people around you in order to be a good human being. The show is brought to you by Indeed, which also sponsors the show. Indeed gives you access to its suite of tools including an AI tool called notion, which allows you to write better, smarter, more complex essays faster than anyone else ever has. In addition, you can do all sorts of other cool stuff with it like create presentations, make notes, edit documents, etc. On top of this, there\'s the notion AI tool, which lets you combine large language models into a productivity note-taking device. You can use it for anything from note taking to working on big projects to tracking your health or finding out about new job opportunities. Afterward, they talk about whether there might actually be intelligent life in the universe. He thinks t